In [3]:
import JSSP
import gym

In [4]:

env_name = "JSSP-v0"
data = [[2,2],[2,1,0,5,1,1,10],[2,1,1,3,1,0,1]]
env = gym.make(env_name, env_data = data)
print("Observation space:", env.observation_space)
print("Action space:", env.action_space)

Observation space: Dict(active:Box([0 0], [1 1], (2,), int64), left_operations:Box([1 1], [2 2], (2,), int64))
Action space: Box([[0 0]
 [0 0]], [[1 2]
 [1 2]], (2, 2), int64)
